# Retrieval-Augmented Generation (RAG)
대규모 문서데이터베이스에서 관련 정보 검색 -> 더 정확하고 상세한 답변 가능


8단계 프로세스   
<사전 준비>
- 1. 도큐먼트 로드(Document Loader): 외부 데이터 소스에서 필요한 **_데이터 로드 및 초기 처리_**
- 2. 텍스트 분할(Text Splitter): 로드된 문서를 **_처리 가능한 작은 단위_(Chunk)** 로 분할
- 3. 임베딩 (Embedding): 문서를 **_벡터 형태_** 로 변환 -> **문서의 의미 수치화**
- 4. 벡터 스토어(Vector Store) 저장: 임베딩된 벡터들을 _데이터베이스에 저장_ 

<런타임>
- 5. 검색기(Retriver): **질문** 주어짐 -> **관련 벡터 검색** in 데이터베이스  ->  유사도 검색(similarity, mmr), Multi-Query, Multi-Retriver
        ```
        Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
        ```
- 6. 프롬프트(Prompt): 검색된 정보를 바탕으로 **언어모델을 위한 질문 구성**
- 7. LLM: 구성된 프롬프트를 사용해 언어모델이 답변 생성, **LLM 모델 정의**
- 8. 체인(Chain): 위 단계를 하나의 _파이프라인_ 으로 묶음

# 01. PDF 문서 기반 QA(Question-Answer)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

### 0. 기본 설정

In [2]:
# skeleton code
from langchain_community.document_loaders import PyMuPDFLoader #PDF-> doc각 페이지
from langchain_text_splitters import RecursiveCharacterTextSplitter #기본 splitter
from langchain_community.vectorstores import FAISS  #벡터 검색 라이브러리
from langchain_core.prompts import PromptTemplate  #(LLM에게 보낼 프롬프트 -> 변수 넣는 템플릿) => 관리,재사용 용이
from langchain_core.runnables import RunnablePassthrough #입력 그대로 전달
from langchain_core.output_parsers import StrOutputParser  #LLM 응답에서 텍스트만 추출 -> str 타입으로 반환
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  #LangChain에 openai를 쉽게 불러올 수 있도록

### 1. Document 로드

In [6]:
# 1. 도큐먼트 로드
# 준비
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
# 단일 pdf파일 -> [페이지 수만큼 Document객체 리스트]
documents = loader.load()

print(f'문서의 페이지 수: {len(documents)}')

문서의 페이지 수: 23


Document 구조
> ```txt
> [
>   Document(1페이지),
>   Document(2페이지),
>   Document(3페이지),
>   ...
> ]
> ```

### 2. 문서 분할

In [ ]:
# 2. 문서 분할
#청크의 최대 글자 수 #겹치는 글자 수->문맥 보존
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
# [페이지 단위 Documents] → [더 작은 chunk Documents]
split_documents = text_splitter.split_documents(documents) # =chunk

print(f'분할된 청크의 수: {len(split_documents)}')


분할된 청크의 수: 43


split_documents 구조
> ```txt
> [
>   Document(1페이지의 chunk1),
>   Document(1페이지의 chunk2),
>   Document(2페이지의 chunk1),
>   Document(3페이지의 chunk1),
>   Document(3페이지의 chunk2),
>   ...
> ]
> ```
> 단순 텍스트 목록, 검색 불가능

### 4. 임베딩

In [11]:
# 3. 임베딩
    # 벡터화
    # chunk를 '의미 기반'의 고차원 벡터로 변환  (의미 기반 검색 o -> 의미유사도  /  키워드 검색 X)
embeddings = OpenAIEmbeddings()

embeddings 구조
> ```txt
> [
>   [0.123, -0.019, 0.772, ... 1536개],
>   [0.551, 0.214, -0.488, ...],
>   ...
> ]
> ```

### 4. 벡터스토어

In [12]:
# 4. Create DB 및 저장
    # 벡터 스토어 생성
    # 벡터들을 FAISS 인덱스에 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)


FAISS 인덱스 구조 (단순 리스트 → 벡터 검색 가능한 DB)
> ```txt
> vector index:
>   [
>     (id=0, vector=chunk1_vector),
>     (id=1, vector=chunk2_vector),
>     (id=2, vector=chunk3_vector),
>     ...
>   ]
> ```
> 청크가 벡터로 저장되어 의미 기반 검색 가능

### 5. Retriver

In [15]:
# 5. Retriver 생성 (문서에 포함되어 있는 정보를 검색하고 생성 = 벡터 DB에서 “질문과 의미적으로 가까운 청크”를 찾아주는 검색기)
retriever = vectorstore.as_retriever()   #FAISS 벡터DB → RAG 검색기


In [11]:
# retriver에 쿼리를 날려, chunk 결과 확인하기
retriever.invoke('메타의 라마에 대해 설명해주세요')

[Document(id='3f75006f-7f53-44f7-b59c-44d35365765d', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 19}, page_content='페이스의  제퍼(Zephyr-7b)가 라마2를 능가\n<갈릴레오의 LLM 환각 지수(RAG 포함 질문과 답변 기준)>\n☞ 출처: Galileo, LLM Hallucination Index, 2023.11.15.'),
 Document(id='aef7d271-67d2-4ea5-a8d9-f35884ceeeea', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'aut

`page_content` → LLM에게 넘겨질 실제 텍스트   
`metadata` → 출처/페이지/청크 번호 정보

### 6. 프롬프트

In [16]:
# 6. 프롬프트 생성
    # LLM이 답변을 생성할 수 있게, _입력 구조를 정형화_하는 단계
prompt = PromptTemplate.from_template( """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question(query): 
{question} 
#Context(response):  
{context} 

#Answer정확하게 답변하세요:""")

`PromptTemplate.from_template()` -> "내가 만든 문자열을 *템플릿*으로 변환해줘"
- 자유 형식 템플릿
- #Question: #Context: 구조는 이렇다고 사람이 이해하기 쉽게 만든 포맷일뿐임
- 템플릿 안에 변수명 필수: `{QUestion}` `{Context}`
 

### 7. LLM

In [17]:
# 7. LLM 생성
llm = ChatOpenAI(model='gpt-5-nano', temperature=0)

### 8. Chain

In [18]:
# 8. Chain 생성
chain = (
    {"context":retriever, "question":RunnablePassthrough()} #사용자 흐름 참고
    | prompt  #최종 프롬프트
    | llm  #llm에 전달 -> 답변 생성
    | StrOutputParser()  #output을 str로 보기좋게
)

사용자 흐름
1. `chain.invoke("메타의 라마 뭐야?")`
2. "question"에는 사용자의 질문이 그대로 들어감
3. → `RunnablePassthrough()`는 “그대로 통과시키기” 역할
4. "context"에는 `retriever`가 검색한 chunk들이 자동으로 들어감

### Test!

In [20]:
# 체인 실행!
    # 문서에 대한 질문 입력 -> 답변 출력
question = "삼성보단 애플이 짱이지"

response = chain.invoke(question)
response

'주어진 컨텍스트에는 애플과 삼성의 비교에 대한 정보가 없습니다. 따라서 “삼성보단 애플이 짱이다”라는 주장에 대해 확인하거나 근거를 제시할 수 없습니다.'

> `question` = 사용자가 입력하는 질문 (자연어 질문)   
> `query` = retriever 입장에서 검색 요청

> `context` = retriever가 찾은 문서 청크   
> `response` = LLM이 만든 최종 답변
